To fix:
- make sure it takes the outlier into account accurately (or skip it!)
- make sure the code isnt bullshit (since chatgpt etc)
- make sure its 5 right and 5 left, sliding window of years
- maybe use a better measure
- make it a more concise set of functions
- make sure no weird words make it through, such as "7In", or find out their context

TESTING CONTEXTS

In [3]:
import pandas as pd
import nltk
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
import csv
import os
import re
from collections import defaultdict

text_dir = r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb"

def extract_year_from_filename(filename):
    match = re.search(r'rst[bl]?_(\d{4})', filename)
    if not match:
        match = re.search(r'rst[bl]?(\d{4})', filename)
    if match:
        return int(match.group(1))
    return None

def read_texts_by_year(directory):
    texts_by_year = defaultdict(list)
    print("Reading text files...")
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            year = extract_year_from_filename(filename)
            if year:
                with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                    text = file.read()
                texts_by_year[year].append(text)
    print("Text files read and organized by year.")
    return texts_by_year

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and len(token) > 1]

def get_collocates_by_sliding_window(texts_by_year, word_of_interest, window_size=5):
    collocates_per_window = {}
    all_years = sorted(texts_by_year.keys())
    start_year = min(all_years)
    end_year = max(all_years)

    print("Analyzing collocates by sliding window...")
    for window_start in range(start_year, end_year - window_size + 2):
        window_end = window_start + window_size - 1
        window_texts = []
        for year in range(window_start, window_end + 1):
            if year in texts_by_year:
                window_texts.extend(texts_by_year[year])
        
        all_tokens = []
        for text in window_texts:
            tokens = word_tokenize(text)
            tokens = [token.lower() for token in tokens if token not in string.punctuation and token.lower() not in stop_words and not token.isdigit()] #removing tokens already here (stopwords, punctuation, digits)
            all_tokens.extend(tokens) 
        
        tokens = lemmatize_tokens(all_tokens) #should allow for seeing different forms of fact, but potentially bad to lemmatize?
        
        total_tokens = len(tokens) #total tokens dont include removed ones; is this bad?
        total_texts = len(window_texts)
        
        bigram_measures = BigramAssocMeasures()
        finder = BigramCollocationFinder.from_words(tokens)
        finder.apply_freq_filter(2) #maybe make it higher to include less (2 is quite low)
        
        collocations = finder.score_ngrams(bigram_measures.pmi)
        
        collocate_stats = []
        collocate_contexts = []

        for bigram, pmi in collocations:
            if word_of_interest in bigram:
                other_word = bigram[0] if bigram[1] == word_of_interest else bigram[1] #dont fully understand this yet, are bigrams 5L 5R?
                observed_freq = finder.ngram_fd[bigram]
                word_freq = finder.word_fd[other_word]
                expected_freq = (finder.word_fd[word_of_interest] * word_freq) / total_tokens
                num_texts = sum(1 for text in window_texts if other_word in text.split())
                
                # Extract contexts with 5 words to the left and right  (BUT THE BIGRAM SHOULD BE 5L 5R)
                for text in window_texts:
                    words = text.split()
                    for i, word in enumerate(words):
                        if word == word_of_interest and other_word in words[max(0, i-5):i+6]:
                            context = ' '.join(words[max(0, i-5):i+6])
                            collocate_contexts.append(f"Years {window_start}-{window_end}: {context}")      

                collocate_stats.append({
                    'word': other_word,
                    'total_corpus': word_freq,
                    'expected_freq': expected_freq,
                    'observed_freq': observed_freq,
                    'num_texts': num_texts,
                    'pmi': pmi
                })
        
        collocate_stats.sort(key=lambda x: x['pmi'], reverse=True)
        collocates_per_window[f"{window_start}-{window_end}"] = {
            'collocates': collocate_stats[:20],
            'total_tokens': total_tokens,
            'total_texts': total_texts,
            'contexts': collocate_contexts
        }
        print(f"Years {window_start}-{window_end}: {len(collocate_stats)} collocates found")

    return collocates_per_window

print("Starting text processing...")
texts_by_year = read_texts_by_year(text_dir)
collocates_by_window = get_collocates_by_sliding_window(texts_by_year, "fact")

# Saving collocation results to CSV files
output_dir = 'collocate_results_rstb'
os.makedirs(output_dir, exist_ok=True)
print("Saving results to CSV files...")

for window, data in collocates_by_window.items():
    filename = os.path.join(output_dir, f'collocates_{window}.csv')
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['No.', 'Word', 'Total no. in window corpus', 'Expected collocate frequency',
                         'Observed collocate frequency', 'In no. of texts', 'Mutual Information value'])
        
        for i, collocate in enumerate(data['collocates'], 1):
            writer.writerow([
                i,
                collocate['word'],
                collocate['total_corpus'],
                f"{collocate['expected_freq']:.2f}",
                collocate['observed_freq'],
                collocate['num_texts'],
                f"{collocate['pmi']:.2f}"
            ])
        
        # Add summary statistics at the end of each file
        writer.writerow([])
        writer.writerow(['Total tokens in window', data['total_tokens']])
        writer.writerow(['Total texts in window', data['total_texts']])

    # Saving collocate contexts to a text file for each window
    context_file = os.path.join(output_dir, f'collocate_contexts_{window}.txt')
    with open(context_file, 'w', encoding='utf-8') as f:
        for context in data['contexts']:
            f.write(context + '\n')

print(f"Collocate analysis completed. Results saved in the '{output_dir}' directory.")
print(f"Contexts saved to separate files for each window in the '{output_dir}' directory.")

Starting text processing...
Reading text files...
Text files read and organized by year.
Analyzing collocates by sliding window...
Years 1887-1891: 243 collocates found
Years 1888-1892: 214 collocates found
Years 1889-1893: 237 collocates found
Years 1890-1894: 320 collocates found
Years 1891-1895: 359 collocates found
Years 1892-1896: 337 collocates found
Years 1893-1897: 357 collocates found


In [ ]:
import pandas as pd
import nltk
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
import csv
import os
import re
from collections import defaultdict

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def extract_year_from_filename(filename):
    match = re.search(r'rst[bl]?_(\d{4})', filename)
    if not match:
        match = re.search(r'rst[bl]?(\d{4})', filename)
    if match:
        return int(match.group(1))
    return None

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and len(token) > 1]

def analyze_collocates(text_dir, output_dir, word_of_interest, window_size=5):
    def read_texts_by_year(directory):
        texts_by_year = defaultdict(list)
        print("Reading text files...")
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                year = extract_year_from_filename(filename)
                if year:
                    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                        text = file.read()
                    texts_by_year[year].append((filename, text))
        print("Text files read and organized by year.")
        return texts_by_year

    def get_collocates_by_sliding_window(texts_by_year):
        collocates_per_window = {}
        all_years = sorted(texts_by_year.keys())
        start_year = min(all_years)
        end_year = max(all_years)

        print("Analyzing collocates by sliding window...")
        for window_start in range(start_year, end_year - window_size + 2):
            window_end = window_start + window_size - 1
            window_texts = []
            for year in range(window_start, window_end + 1):
                if year in texts_by_year:
                    window_texts.extend(texts_by_year[year])
            
            all_tokens = []
            for _, text in window_texts:
                tokens = word_tokenize(text)
                tokens = [token.lower() for token in tokens if token not in string.punctuation and token.lower() not in stop_words and not token.isdigit()] 
                all_tokens.extend(tokens)
            
            tokens = lemmatize_tokens(all_tokens)
            
            total_tokens = len(tokens)
            total_texts = len(window_texts)
            
            bigram_measures = BigramAssocMeasures()
            finder = BigramCollocationFinder.from_words(tokens)
            finder.apply_freq_filter(2)
            
            collocations = finder.score_ngrams(bigram_measures.pmi)
            
            collocate_stats = []
            collocate_contexts = defaultdict(list)

            for bigram, pmi in collocations:
                if word_of_interest in bigram:
                    other_word = bigram[0] if bigram[1] == word_of_interest else bigram[1]
                    observed_freq = finder.ngram_fd[bigram]
                    word_freq = finder.word_fd[other_word]
                    expected_freq = (finder.word_fd[word_of_interest] * word_freq) / total_tokens
                    num_texts = sum(1 for _, text in window_texts if other_word in text.split())
                    
                    # Extract contexts with 5 words to the left and right
                    for filename, text in window_texts:
                        words = word_tokenize(text)
                        for i in range(len(words)):
                            if words[i].lower() == word_of_interest.lower():
                                start = max(0, i - 5)
                                end = min(len(words), i + 6)
                                context = words[start:end]
                                if other_word.lower() in [w.lower() for w in context]:
                                    context_str = ' '.join(context)
                                    collocate_contexts[filename].append({
                                        'collocate': other_word,
                                        'context': context_str
                                    })

                    collocate_stats.append({
                        'word': other_word,
                        'total_corpus': word_freq,
                        'expected_freq': expected_freq,
                        'observed_freq': observed_freq,
                        'num_texts': num_texts,
                        'pmi': pmi
                    })
            
            collocate_stats.sort(key=lambda x: x['pmi'], reverse=True)
            collocates_per_window[f"{window_start}-{window_end}"] = {
                'collocates': collocate_stats[:20],
                'total_tokens': total_tokens,
                'total_texts': total_texts,
                'contexts': collocate_contexts
            }
            print(f"Years {window_start}-{window_end}: {len(collocate_stats)} collocates found")

        return collocates_per_window

    texts_by_year = read_texts_by_year(text_dir)
    collocates_by_window = get_collocates_by_sliding_window(texts_by_year)

    # Saving collocation results to CSV files
    os.makedirs(output_dir, exist_ok=True)
    print("Saving results to CSV files...")

    for window, data in collocates_by_window.items():
        filename = os.path.join(output_dir, f'collocates_{window}.csv')
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['No.', 'Word', 'Total no. in window corpus', 'Expected collocate frequency',
                             'Observed collocate frequency', 'In no. of texts', 'Mutual Information value'])
            
            for i, collocate in enumerate(data['collocates'], 1):
                writer.writerow([
                    i,
                    collocate['word'],
                    collocate['total_corpus'],
                    f"{collocate['expected_freq']:.2f}",
                    collocate['observed_freq'],
                    collocate['num_texts'],
                    f"{collocate['pmi']:.2f}"
                ])
            
            # Add summary statistics at the end of each file
            writer.writerow([])
            writer.writerow(['Total tokens in window', data['total_tokens']])
            writer.writerow(['Total texts in window', data['total_texts']])

        # Saving collocate contexts to separate CSV files for each text
        contexts_dir = os.path.join(output_dir, f'contexts_{window}')
        os.makedirs(contexts_dir, exist_ok=True)
        
        for filename, contexts in data['contexts'].items():
            context_file = os.path.join(contexts_dir, f'contexts_{os.path.splitext(filename)[0]}.csv')
            with open(context_file, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Filename', 'Collocate', 'Context'])
                for context in contexts:
                    writer.writerow([filename, context['collocate'], context['context']])

    print(f"Collocate analysis completed. Results saved in the '{output_dir}' directory.")
    print(f"Contexts saved to separate CSV files for each text in subdirectories of '{output_dir}'.")

analyze_collocates(r"D:\Fact_fiction_corpus\texts\royal society\txt_rstb", "collocate_results_rstb_latest", "fact")
analyze_collocates(r"D:\Fact_fiction_corpus\texts\royal society\txt_rstl", "collocate_results_rstl_latest", "fact")
